In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls /kaggle/input


Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 262 (delta 126), reused 221 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 1.13 MiB | 8.63 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/kaggle/working/NIDS_Coursework/My_Code
UNSW-NB15_combined.csv


In [2]:
import os
import random
import numpy as np
import tensorflow as tf

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(42)

2025-08-27 13:43:11.252223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756302191.438918      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756302191.495284      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
import numpy as np

path = "/kaggle/input/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')



# --------------------------- perform preprocessing--------------

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)

# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)

# Filter out categorical one-hot encoded columns
numerical_columns = [col for col in processed_full_df 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_") or
                             col == 'label')]

target = processed_full_df['label']
features = processed_full_df.drop(columns=['label'])

(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X = features.astype("float32").values  # (257673, 95) as float32
y = target.astype("int32").values      # (257673,)


# reshape for (samples, timesteps, features=1)
X = np.expand_dims(X, axis=-1)  # (257673, 95, 1)

# Split train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")



input_shape = (X_train.shape[1], X_train.shape[2])


X_train: (206138, 95, 1)
y_train: (206138,)
X_val: (41228, 95, 1)
y_val: (41228,)
X_test: (10307, 95, 1)
y_test: (10307,)


In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        numerator = tp * tn - fp * fn
        denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        mcc = numerator / (denominator + 1e-10)
        specificity = tn / (tn + fp + 1e-10)
        balanced_acc = (recall + specificity) / 2
    else:
        mcc = 0
        balanced_acc = 0
        specificity = 0

    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc': mcc,
        'specificity': specificity,
        'balanced_accuracy': balanced_acc
    }

## Model Building

In [6]:
from tensorflow.keras import layers, models

# Bidirectional LSTM model
model = models.Sequential([
    layers.Bidirectional(layers.LSTM(64, return_sequences=False), input_shape=(95,1)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_bi_lstm = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_data=(X_val, y_val),
    verbose=1
)

# Predictions
y_pred_bi_lstm = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate
bi_lstm_results = evaluate_metrics(y_true=y_test, y_pred=y_pred_bi_lstm)
print(f"Bidirectional LSTM results: {bi_lstm_results}")


I0000 00:00:1756302209.068509      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756302209.069165      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


2025-08-27 13:43:30.714108: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
I0000 00:00:1756302214.032087      94 cuda_dnn.cc:529] Loaded cuDNN version 90300


803/806 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7121 - loss: 0.5344

2025-08-27 13:43:43.896340: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.7124 - loss: 0.5339 - val_accuracy: 0.8718 - val_loss: 0.3082
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8795 - loss: 0.2805 - val_accuracy: 0.9003 - val_loss: 0.2246
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8968 - loss: 0.2302 - val_accuracy: 0.8979 - val_loss: 0.2268
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9000 - loss: 0.2140 - val_accuracy: 0.9007 - val_loss: 0.2053
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9021 - loss: 0.2034 - val_accuracy: 0.9050 - val_loss: 0.2060
Epoch 6/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8990 - loss: 0.2160 - val_accuracy: 0.8990 - val_loss: 0.1922
Epoch 7/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9056 - loss: 0.1897 - val_accuracy: 0.9099 - val_loss: 0.1779
Epoch 8/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9062 - loss: 0.1856 - val_accurac

2025-08-27 13:46:55.352742: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Bidirectional LSTM results: {'f1': 0.9307509881422924, 'precision': 0.970971466270823, 'recall': 0.8937300743889479, 'mcc': 0.8259786978136132, 'specificity': 0.9526881720429852, 'balanced_accuracy': 0.9232091232159665}


## Attack codes

In [7]:
import tensorflow as tf

@tf.function
def fgsm_attack_tf(model, inputs, labels, epsilon=0.05, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        predictions = model(inputs, training=False)
        loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        loss = tf.reduce_mean(loss)

    gradients = tape.gradient(loss, inputs)
    signed_grad = tf.sign(gradients)
    x_adv = inputs + epsilon * signed_grad
    x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)
    return x_adv


@tf.function
def pgd_attack_tf(model, inputs, labels, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)

    for _ in tf.range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(gradients)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


@tf.function
def mi_fgsm_attack(model, inputs, labels, epsilon=0.05, alpha=0.01, steps=40, decay=1.0, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)
    g = tf.zeros_like(inputs)

    for _ in tf.range(steps):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        # Normalize gradient
        grad_norm = tf.reduce_mean(tf.abs(grad), axis=list(range(1, len(grad.shape))), keepdims=True)
        grad = grad / (grad_norm + 1e-8)

        g = decay * g + grad
        x_adv = x_adv + alpha * tf.sign(g)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


In [8]:
def evaluate_model(model, X, y,epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ---------------- Clean Evaluation ----------------
    print("✅ Clean Evaluation:")
    y_pred_clean = (model.predict(x_clean_tf) > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"  F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- FGSM Evaluation ----------------
    print("\nFGSM Attack Evaluation")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- PGD Evaluation ----------------
    print("\nPGD Attack Evaluation:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- MI-FGSM Evaluation ----------------
    print("\nMI-FGSM Attack Evaluation:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10   
        decay = 1.0   

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f" MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results
        


In [9]:
print("CNN Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)



CNN Results: 
✅ Clean Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  F1: 0.9308, Precision: 0.9710, Recall: 0.8937, MCC: 0.8259786978136132, Specificity: 0.9526881720429852, Balanced Acc: 0.9232091232159665

FGSM Attack Evaluation
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.01 → F1: 0.7930, Precision: 0.8433, Recall: 0.7483, MCC: 0.4866424805408501, Specificity: 0.7537634408601948, Balanced Acc: 0.7510277656707229
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 13:47:02.442230: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.03 → F1: 0.8141, Precision: 0.7369, Recall: 0.9094, MCC: 0.39336084319956405, Specificity: 0.42499999999998855, Balanced Acc: 0.6671834674358528
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.05 → F1: 0.7341, Precision: 0.6463, Recall: 0.8494, MCC: 0.034423016976473084, Specificity: 0.17688172043010278, Balanced Acc: 0.5131410272106488
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.07 → F1: 0.6921, Precision: 0.6111, Recall: 0.7978, MCC: -0.1306398202667072, Specificity: 0.10107526881720158, Balanced Acc: 0.44942939089865697

PGD Attack Evaluation:
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 13:47:13.737357: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  PGD ε=0.01 → F1: 0.4872, Precision: 0.6897, Recall: 0.3767, MCC: 0.07723275551889602, Specificity: 0.6999999999999812, Balanced Acc: 0.5383254896007192
 45/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.03 → F1: 0.2252, Precision: 0.2960, Recall: 0.1817, MCC: -0.5740299543415783, Specificity: 0.2346774193548324, Balanced Acc: 0.2081994960748657
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 13:47:52.973020: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.05 → F1: 0.1308, Precision: 0.1653, Recall: 0.1082, MCC: -0.8367819214408007, Specificity: 0.032258064516128164, Balanced Acc: 0.07025078722997846
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 13:47:59.493668: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.07 → F1: 0.1801, Precision: 0.2159, Recall: 0.1545, MCC: -0.8093281190608349, Specificity: 0.005913978494623497, Balanced Acc: 0.08023040658449104


## Adversarial Training

In [10]:
import tensorflow as tf
from tensorflow.keras import Model

class PGDAdversarialTrainerCNN(tf.keras.Model):
    def __init__(self, base_model, epsilon_range=(0.01, 0.1), alpha=0.01, num_iter=7, clip_min=-1.5, clip_max=1.5):
        super().__init__()
        self.base_model = base_model
        self.epsilon_range = epsilon_range
        self.alpha = alpha
        self.num_iter = num_iter
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.metric = tf.keras.metrics.BinaryAccuracy()

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer

    def pgd_attack_batch(self, x, y):
        x_adv = tf.identity(x)
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        epsilon = tf.random.uniform([], *self.epsilon_range)

        for _ in range(self.num_iter):
            with tf.GradientTape() as tape:
                tape.watch(x_adv)
                preds = self.base_model(x_adv, training=True)
                loss = self.loss_fn(y, preds)

            grad = tape.gradient(loss, x_adv)
            x_adv = x_adv + self.alpha * tf.sign(grad)
            # Project back to epsilon ball around original input
            x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
            x_adv = tf.clip_by_value(x_adv, self.clip_min, self.clip_max)

        return x_adv

    def train_step(self, data):
        x, y = data
        x_adv = self.pgd_attack_batch(x, y)

        # Combine clean and adversarial samples
        x_combined = tf.concat([x, x_adv], axis=0)
        y_combined = tf.concat([y, y], axis=0)
        y_combined = tf.expand_dims(tf.cast(y_combined, tf.float32), axis=-1)

        with tf.GradientTape() as tape:
            preds = self.base_model(x_combined, training=True)
            loss = self.loss_fn(y_combined, preds)

        grads = tape.gradient(loss, self.base_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.base_model.trainable_variables))
        self.metric.update_state(y_combined, preds)

        return {"loss": loss, "accuracy": self.metric.result()}

    def test_step(self, data):
        x, y = data
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        preds = self.base_model(x, training=False)
        loss = self.loss_fn(y, preds)
        self.metric.update_state(y, preds)
        return {"loss": loss, "accuracy": self.metric.result()}

    def call(self, inputs):
        return self.base_model(inputs)


In [11]:
# Wrap your existing CNN

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',   # monitor validation loss
    patience=3,           # stop if no improvement for 3 epochs
    restore_best_weights=True
)


adv_model = PGDAdversarialTrainerCNN(
    base_model=model,           
    epsilon_range=(0.01, 0.05),
    alpha=0.01,
    num_iter=7,
    clip_min=-1.5,
    clip_max=1.5
)

adv_model.compile(optimizer=tf.keras.optimizers.Adam())

# Train with adversarial training
adv_model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=20,
    callbacks=[early_stop] 

)

print("Advesarial training Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=adv_model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)


Epoch 1/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8435 - loss: 0.3139

2025-08-27 13:49:19.119939: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 79s 85ms/step - accuracy: 0.8435 - loss: 0.3139 - val_accuracy: 0.8924 - val_loss: 0.1160
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 68s 84ms/step - accuracy: 0.8881 - loss: 0.2527 - val_accuracy: 0.8958 - val_loss: 0.0806
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8895 - loss: 0.2251 - val_accuracy: 0.9143 - val_loss: 0.0581
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8906 - loss: 0.2021 - val_accuracy: 0.9171 - val_loss: 0.0545
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8961 - loss: 0.1886 - val_accuracy: 0.9175 - val_loss: 0.0516
Epoch 6/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8992 - loss: 0.1814 - val_accuracy: 0.9066 - val_loss: 0.0597
Epoch 7/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8814 - loss: 0.2738 - val_accuracy: 0.8952 - val_loss: 0.1073
Epoch 8/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.8908 - loss: 0.2358 - val_accurac

2025-08-27 13:57:11.451747: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  F1: 0.9341, Precision: 0.9649, Recall: 0.9053, MCC: 0.8301445380836768, Specificity: 0.9416666666666413, Balanced Acc: 0.9234673093466803

FGSM Attack Evaluation
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  FGSM ε=0.01 → F1: 0.9288, Precision: 0.9589, Recall: 0.9004, MCC: 0.8157983204226152, Specificity: 0.9317204301075018, Balanced Acc: 0.9160651641960008
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.03 → F1: 0.9156, Precision: 0.9458, Recall: 0.8874, MCC: 0.7814940154924713, Specificity: 0.9099462365591153, Balanced Acc: 0.8986500577057001
 46/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 13:58:06.854708: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 MI-FGSM ε=0.05 → F1: 0.8322, Precision: 0.8953, Recall: 0.7774, MCC: 0.5957017400297584, Specificity: 0.8389784946236334, Balanced Acc: 0.8082094537791007
 45/323 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-08-27 13:58:13.346238: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 MI-FGSM ε=0.07 → F1: 0.7750, Precision: 0.8522, Recall: 0.7106, MCC: 0.47396090402249563, Specificity: 0.7817204301075058, Balanced Acc: 0.7461813020432777


In [12]:
# generate adversarial values for gan training

# experimental PGD code (GPU-optimized, batched)
def pgd_attack_tf_fast(model, x, y, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.reshape(tf.cast(y, tf.float32), (-1, 1))
    x_orig = tf.identity(x)
    x_adv = tf.identity(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(y, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x_orig - epsilon, x_orig + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv

def batch_pgd_attack(model, X, y, attack_fn, batch_size=1024, **kwargs):
    adv_list = []
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        x_adv_batch = attack_fn(model, X_batch, y_batch, **kwargs)
        adv_list.append(x_adv_batch)
    return tf.concat(adv_list, axis=0)

# Generate adversarial examples in batches to leverage GPU
X_adv_train = batch_pgd_attack(model, X_train, y_train, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_train shape: {X_adv_train.shape}")

X_adv_val = batch_pgd_attack(model, X_val, y_val, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_val shape: {X_adv_val.shape}")


X_adv_train shape: (206138, 95, 1)
X_adv_val shape: (41228, 95, 1)


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping

# ------------------------------
# Generator CNN (1D output)
# ------------------------------
class GeneratorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1D(128, 3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        # final output: 1 channel (same as clean input)
        self.output_layer = layers.Conv1D(1, 3, padding='same', activation='linear')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        return self.output_layer(x)


# ------------------------------
# Discriminator CNN (1D input)
# ------------------------------
class DiscriminatorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.global_pool = layers.GlobalAveragePooling1D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.global_pool(x)
        x = self.dense1(x)
        return self.output_layer(x)


# ------------------------------
# GAN Purifier CNN
# ------------------------------
class GANPurifierCNN(Model):
    def __init__(self, generator, discriminator, lambda_adv=0.5):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_adv = lambda_adv
        self.bce = tf.keras.losses.BinaryCrossentropy()
        self.l1 = tf.keras.losses.MeanAbsoluteError()
        self.d_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.g_optimizer = tf.keras.optimizers.Adam(1e-4)

    def compile(self, **kwargs):
        super().compile(**kwargs)

    def train_step(self, data):
        x_clean, x_adv = data
        with tf.GradientTape(persistent=True) as tape:
            x_fake = self.generator(x_adv, training=True)
            real_pred = self.discriminator(x_clean, training=True)
            fake_pred = self.discriminator(x_fake, training=True)

            # Discriminator loss
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)

            # Generator loss (reconstruction + adversarial)
            g_loss = self.l1(x_clean, x_fake) + \
                     self.lambda_adv * self.bce(tf.ones_like(fake_pred), fake_pred)

            # Cosine similarity
            x_clean_flat = tf.reshape(x_clean, (tf.shape(x_clean)[0], -1))
            x_fake_flat = tf.reshape(x_fake, (tf.shape(x_fake)[0], -1))
            cos_sim = -tf.reduce_mean(
                tf.keras.losses.cosine_similarity(x_clean_flat, x_fake_flat)
            )

        # Compute gradients
        d_grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        g_grads = tape.gradient(g_loss, self.generator.trainable_variables)

        # Apply gradients
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss,
                "reconstruction_loss": self.l1(x_clean, x_fake),
                "cosine_similarity": cos_sim}





In [14]:
# ------------------------------
# Instantiate models
# ------------------------------
generator = GeneratorCNN()
discriminator = DiscriminatorCNN()

# Build models with dummy input (1 channel)
dummy_input = tf.zeros((1, 95, 1))
generator(dummy_input)
discriminator(dummy_input)

gan_cnn = GANPurifierCNN(generator, discriminator, lambda_adv=0.5)
gan_cnn.compile()

# ------------------------------
# Prepare dataset
# ------------------------------
batch_size = 512
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_adv_train))
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

early_stop = EarlyStopping(
    monitor="cosine_similarity",
    mode="max",              # higher cosine similarity is better
    patience=10,
    min_delta=1e-4,
    restore_best_weights=True,
    verbose=1
)

# ------------------------------
# Train
# ------------------------------
gan_cnn.fit(train_dataset, epochs=100, callbacks=[early_stop])


Epoch 1/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - cosine_similarity: 0.8928 - d_loss: 1.3789 - g_loss: 0.4366 - reconstruction_loss: 0.0878
Epoch 2/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - cosine_similarity: 0.9803 - d_loss: 1.3803 - g_loss: 0.3841 - reconstruction_loss: 0.0356
Epoch 3/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - cosine_similarity: 0.9867 - d_loss: 1.3794 - g_loss: 0.3775 - reconstruction_loss: 0.0290
Epoch 4/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - cosine_similarity: 0.9893 - d_loss: 1.3774 - g_loss: 0.3756 - reconstruction_loss: 0.0267
Epoch 5/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9907 - d_loss: 1.3717 - g_loss: 0.3754 - reconstruction_loss: 0.0251
Epoch 6/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9917 - d_loss: 1.3587 - g_loss: 0.3773 - reconstruction_loss: 0.0238
Epoch 7/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - cosine_similarity: 0.9923 - d_loss: 1.3311 - g_loss: 0.3841 -

In [15]:
# Generate purified outputs
x_purified_val = generator(X_adv_val, training=False)

# Cast to float32 for evaluation
x_purified_val = tf.cast(x_purified_val, tf.float32)
X_val_cast = tf.cast(X_val, tf.float32)

# L1 (Mean Absolute Error)
l1_loss = tf.reduce_mean(tf.abs(X_val_cast - x_purified_val))

# L2 (Mean Squared Error)
l2_loss = tf.reduce_mean(tf.square(X_val_cast - x_purified_val))

# Normalize along sequence dimension
X_val_norm = tf.nn.l2_normalize(tf.squeeze(X_val_cast, axis=-1), axis=1)           
x_purified_norm = tf.nn.l2_normalize(tf.squeeze(x_purified_val, axis=-1), axis=1)  

# Cosine similarity
cos_sim = tf.reduce_mean(tf.reduce_sum(X_val_norm * x_purified_norm, axis=1))

print(f"🧪 Purifier Evaluation Metrics:")
print(f"✅ Cosine Similarity: {cos_sim:.6f}")
print(f"   ➤ L1 Loss (MAE): {l1_loss:.6f}")
print(f"   ➤ L2 Loss (MSE): {l2_loss:.6f}")


W0000 00:00:1756303404.060232      36 gpu_utils.cc:68] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.


🧪 Purifier Evaluation Metrics:
✅ Cosine Similarity: 0.992749
   ➤ L1 Loss (MAE): 0.022486
   ➤ L2 Loss (MSE): 0.001535


## GAN+Adv Training evaluation

In [16]:
def evaluate_model(model, generator, adv_model, X, y, epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ----------- Clean Data Evaluation (no adversarial) -----------
    print("✅ Clean Evaluation (GAN purified):")
    x_purified_clean = generator(x_clean_tf, training=False)
    y_pred_clean_prob = adv_model.predict(x_purified_clean)
    y_pred_clean = (y_pred_clean_prob > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"Clean → F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- FGSM Attack Evaluation -----------
    print("\nFGSM Attack Evaluation with GAN Purification")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- PGD Attack Evaluation -----------
    print("\nPGD Attack Evaluation with GAN Purification:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

      # ----------- MI-FGSM Attack Evaluation -----------
    print("\nMI-FGSM Attack Evaluation with GAN Purification:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10
        decay = 1.0

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results


In [17]:
fgsm_full, pgd_full, mi_full = evaluate_model(model= model,generator=generator,adv_model= model, X = X_test, y = y_test)


✅ Clean Evaluation (GAN purified):
 44/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:03:24.897095: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Clean → F1: 0.9304, Precision: 0.9448, Recall: 0.9165, MCC: 0.8129309523798878, Specificity: 0.905107526881696, Balanced Acc: 0.910804864093649

FGSM Attack Evaluation with GAN Purification
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.01 → F1: 0.9279, Precision: 0.9409, Recall: 0.9153, MCC: 0.8055356895416232, Specificity: 0.8981182795698683, Balanced Acc: 0.9067029837199577
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.03 → F1: 0.9209, Precision: 0.9329, Recall: 0.9092, MCC: 0.7861547246802368, Specificity: 0.8841397849462128, Balanced Acc: 0.8966774528192427
 44/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:03:30.213976: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.05 → F1: 0.9095, Precision: 0.9216, Recall: 0.8977, MCC: 0.7553892853669503, Specificity: 0.8647849462365359, Balanced Acc: 0.8812310946455186
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  FGSM ε=0.07 → F1: 0.8829, Precision: 0.9021, Recall: 0.8646, MCC: 0.6885419251811344, Specificity: 0.833870967741913, Balanced Acc: 0.8492263598387719

PGD Attack Evaluation with GAN Purification:
 45/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

...

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.03 → F1: 0.9203, Precision: 0.9316, Recall: 0.9094, MCC: 0.7843074906697098, Specificity: 0.8817204301075031, Balanced Acc: 0.8955436824896101
 45/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:04:07.538438: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  MI-FGSM ε=0.05 → F1: 0.9084, Precision: 0.9213, Recall: 0.8959, MCC: 0.7529012024098487, Specificity: 0.8645161290322348, Balanced Acc: 0.8801858009667018
 45/323 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-08-27 14:04:12.622871: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
  MI-FGSM ε=0.07 → F1: 0.8759, Precision: 0.9050, Recall: 0.8486, MCC: 0.6770682496894163, Specificity: 0.8422043010752461, Balanced Acc: 0.8454227820846096
